In [0]:
# Biblioteca
from pyspark.sql import functions as F, types as T
import re
import os
from datetime import datetime


In [0]:
# Criação de variaveis: catalog, schema, volume, local da camada bronze, onde será silver
CATALOG = "bh"
SCHEMA  = "mobilidade"
VOLUME  = "data_mobility_bh"

BASE_PATH         = f"/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}"
BRONZE_PARQUET    = f"{BASE_PATH}/bronze/parquet"      
SILVER_DELTA_DIR  = f"{BASE_PATH}/silver/mco"          
CHECKPOINTS_DIR   = f"{BASE_PATH}/checkpoints/silver_mco"  

In [0]:
#Renomear colunas e manter um padrão.
COLUMN_RENAMES = {
    "Linha": "linha",
    "SENTIDO": "sentido",
    "data_hora": "data_hora",  
    "Latitude": "latitude",
    "Longitude": "longitude",
    "Placa": "placa",
}

In [0]:
# Gravar colunas que não pode ser null 
CRITICAL_NOT_NULL = ["data_hora", "latitude", "longitude"]

# Chave para verificar registros duplicados
DEDUP_KEYS = ["linha", "data_hora", "latitude", "longitude"]

In [0]:
#Consumi camada bronze
df_bronze = (
    spark.read
        .option("header", "true")
        .option("inferSchema", "true")
        .option("delimiter", ";")   
        .csv(BRONZE_PARQUET)             
)



In [0]:
# tipos timestamp, double, string
df = cast_columns(df_bronze)

In [0]:
# Cria colunas de tempo úteis
df = enrich_time_columns(df)

In [0]:
# Adiciona colunas de linhagem
df = add_lineage_columns(df, BRONZE_PARQUET)

In [0]:
# Remover registros duplicidados
if all([c in df.columns for c in DEDUP_KEYS]):
    qtd_registros = df.count()
    df = df.dropDuplicates(DEDUP_KEYS)
    qtd_RegistrosApos = df.count()

In [0]:

# Verifica diretório
os.makedirs(SILVER_DELTA_DIR, exist_ok=True)
(df.write
   .format("delta")
   .mode("overwrite")
   .save(SILVER_DELTA_DIR)
)


In [0]:

 #Salva a camada silver
(df.write
   .format("delta")
   .mode("overwrite")
   .save(SILVER_DELTA_DIR)
)
